# Choose a similar subway station when moving in BeiJing
### Li Huibin
### 2021-01-20

# 1. Introduction
Because of the change of work location if you living in **Beijing**, we need to move frequently about once every two years.

When we move, we have to choose a similar community or living environment to reduce the difference caused by moving. 
Therefore, if we use the poly in a similar way, it would be a very good idea to aggregate business information around the subway station and find a new subway station as a moving location.


# 2. Business Problem
The purpose is to help working people choose their ideal living environment to reduce the incompatibility caused by moving or traveling.

It can also help merchants choose the best place to run a restaurant, hotel or other entertainment venue

# 3. Data Description
We need the location data of Beijing subway stations. Due to government regulations, the name of each subway station is unique. We download and organize them through the list in wikipedia.

Obtain the latitude and longitude information of each subway station through Amap instead of FourSquare, because Amap has a better performance in China. 
You can get more detailed information about amap [here](https://lbs.amap.com/)

Through FourSquare, we will get information about businesses around the subway station.

In [80]:
import requests
import pandas as pd
from pandas import DataFrame as Df
from geopy.geocoders import Nominatim
!pip install folium
import folium
import numpy as np

In [1]:
CLIENT_ID = 'DBFDCN0X3EAH5JQ0HTTUVDVGRYZ1FBYUJLI3WE4YO5U003LR' # your Foursquare ID
CLIENT_SECRET = 'DL2SKJBFLQSQL4ZDR2YFJIP3GQUHS1KARHFVK50LLQ4MUSXA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBFDCN0X3EAH5JQ0HTTUVDVGRYZ1FBYUJLI3WE4YO5U003LR
CLIENT_SECRET:DL2SKJBFLQSQL4ZDR2YFJIP3GQUHS1KARHFVK50LLQ4MUSXA


### Download station table from wiki

In [25]:
station_list = pd.read_html("https://zh.wikipedia.org/wiki/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E8%BD%A6%E7%AB%99%E5%88%97%E8%A1%A8")

In [57]:
station_set = set()
for stations in station_list[1: 24]:
    station_set |= set(stations.loc[:, stations.columns[0]].to_list())
station_df = Df(station_set)
station_df = station_df.dropna()
station_df = station_df.rename(columns = {0:"name"})
station_df

,name
1,上地站
2,平安里站
3,大红门南站
4,甘家口站
5,惠新西街北口站
...,...
351,北工大西门站
352,海淀五路居站
353,花园桥站
354,六道口站


### I have search lon and lat by amap api

In [58]:
subway_lon_lat = pd.read_csv("http://favormylikes.com/subway_lon_lat.csv")

In [67]:
subway_lon_lat = subway_lon_lat[["name", "lon", "lat"]]
subway_lon_lat = subway_lon_lat.dropna()
subway_lon_lat

,name,lon,lat
0,稻田站,116.222049,39.781599
1,东夏园站,116.478299,39.922750
2,蒲黄榆站,116.423826,39.865875
3,五道口站,116.337582,39.992683
4,经海一路站（京东方）,116.546640,39.784002
...,...,...,...
350,北土城站,116.405083,39.976917
351,关庄站,116.430500,40.000250
352,三元桥站,116.456540,39.961100
353,东大桥站,116.450997,39.922745


In [68]:
map_beijing = folium.Map(location=[39.9042, 116.4074], zoom_start=10)
# add markers to map
for lat, lng, name in zip(subway_lon_lat['lat'], subway_lon_lat['lon'], subway_lon_lat['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_beijing) 
    
map_beijing

In [2]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [69]:
venues = getNearbyVenues(names=subway_lon_lat['name'],
                        latitudes=subway_lon_lat['lat'],
                        longitudes=subway_lon_lat['lon']
                        )

稻田站
东夏园站
蒲黄榆站
五道口站
经海一路站（京东方）
马家堡站
湾子站
苹果园站
牡丹园站
知春里站
亦庄火车站
后沙峪站
泥洼站
北苑站
丰台站
巴沟站
大钟寺站
肖村站
和平门站
林萃桥站
四惠东站
金台路站
南锣鼓巷站
动物园站
国家图书馆站
青年路站
石榴庄站
大葆台站
大郊亭站
草房站
梨园站
西局站
马连洼站
育知路站
芍药居站
生命科学园站
西黄村站
亦庄桥站
定海园站
八里桥站
黑庄户站
俸伯站
鼓楼大街站
菜市口站
北京南站
国贸站
3号航站楼站
次渠南站
光熙门站
孙河站
西四站
霍营站
南楼梓庄站
高米店南站
枣园站
知春路站
磁器口站
大望路站
车公庄西站
北京站
望京南站
昌平西山口站
义和庄站
白盆窑站
花庄站
栗园庄站
公益西桥站
双桥站
垡头站
望京西站
刘家窑站
北安河站
什刹海站
东管头南站
丰台科技园站
善各庄站
黄村火车站
四道桥站
临河里站
万源街站
阎村站
亮马桥站
望京东站
亦庄文化园站
广渠门外站
旧宫站
人民大学站
立水桥南站
东四站
大石河东站
安定门站
上地站
将台站
四海庄站
复兴门站
土桥站
西土城站
立水桥站
四惠站
莲花桥站
2号航站楼站
西二旗站（老站）[注 5]
成寿寺站
草桥站
宣武门站
景泰站
安河桥北站
潞城站
次渠站
黄渠站
奥林匹克公园站
朝阳门站
惠新西街南口站
阜成门站
亦创会展中心站
灵境胡同站
郭庄子站
万寿寺站
回龙观站
七里庄站
昌平站
泰河路站
首经贸站
永丰站
东湖渠站
北运河东站
九棵树站
健德门站
东风北桥站
团结湖站
北沙滩站
黄村西大街站
双井站
海户屯站
颐和园西门站
大兴新城站
顺义站
中国美术馆站
大红门站
昌平东关站
广渠门内站
大屯路东站
丰台南路站
苏州桥站
九龙山站
清河站
崔各庄站
清华东路西口站
魏公村站
五福堂站
良乡大学城西站
通州北关站
瀛海站
农业展览馆站
沙河高教园站
老观里站
屯佃站
火器营站
海淀黄庄站
六里桥站
天通苑站
高家园站
六道口站
北工大西门站
和平里北街站
十里堡站
北邵洼站
西二旗站（新站）
十三陵景区站
宋家庄站
西二旗站
田村站
十里河站
郭公庄站
崇文门站
生物医药基地站
奥体中心站
回龙观东大街站
欢乐谷景区站
分钟寺站
苏州街站
饶乐府站
天宫院站
管庄站
平西府站
望京站
大红门南站
焦化厂站
篱笆房站

In [70]:
print(venues.shape)
venues.head()

(2088, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,稻田站,39.781599,116.222049,2015第四届国际兰花大会,39.781565,116.218991,Botanical Garden
1,东夏园站,39.922750,116.478299,健一公馆 JE Mansion,39.925264,116.482269,Hotel
2,东夏园站,39.922750,116.478299,羊大爷涮肉,39.921710,116.480619,Hotpot Restaurant
3,东夏园站,39.922750,116.478299,McDonald's (麦当劳),39.921828,116.473882,Fast Food Restaurant
4,东夏园站,39.922750,116.478299,Crayfish Master,39.924402,116.478695,Seafood Restaurant


In [72]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
2号航站楼站,5,5,5,5,5,5
3号航站楼站,9,9,9,9,9,9
七里庄站,5,5,5,5,5,5
万寿寺站,3,3,3,3,3,3
万泉河桥站,31,31,31,31,31,31
...,...,...,...,...,...,...
黄村火车站,1,1,1,1,1,1
黄村西大街站,3,3,3,3,3,3
黄渠站,4,4,4,4,4,4


In [73]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 212 uniques categories.


In [74]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vietnamese Restaurant,Watch Shop,Water Park,Whisky Bar,Wings Joint,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant,Zoo,Zoo Exhibit
0,稻田站,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,东夏园站,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,东夏园站,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,东夏园站,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,东夏园站,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
onehot.shape

(2088, 213)

In [76]:
beijing_grouped = onehot.groupby('Neighborhood').mean().reset_index()
beijing_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vietnamese Restaurant,Watch Shop,Water Park,Whisky Bar,Wings Joint,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant,Zoo,Zoo Exhibit
0,2号航站楼站,0.0,0.0,0.000000,0.2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,3号航站楼站,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,七里庄站,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,万寿寺站,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,万泉河桥站,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.032258,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,黄村火车站,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
280,黄村西大街站,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
281,黄渠站,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
282,鼓楼大街站,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [77]:
beijing_grouped.shape

(284, 213)

In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = beijing_grouped['Neighborhood']

for ind in np.arange(beijing_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(beijing_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2号航站楼站,Airport Terminal,Coffee Shop,Duty-free Shop,Chinese Restaurant,Café,Zoo Exhibit,Fabric Shop,Food Truck,Food Court,Food & Drink Shop
1,3号航站楼站,Coffee Shop,Fast Food Restaurant,Ice Cream Shop,Airport,New American Restaurant,Pizza Place,Bakery,Zoo Exhibit,Fabric Shop,Food Court
2,七里庄站,Metro Station,Shopping Mall,Hotpot Restaurant,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
3,万寿寺站,Korean Restaurant,Halal Restaurant,Electronics Store,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio,Fast Food Restaurant
4,万泉河桥站,Chinese Restaurant,Café,Coffee Shop,College Cafeteria,Bookstore,Fast Food Restaurant,Convenience Store,Stadium,Cantonese Restaurant,Gym / Fitness Center


In [82]:
from sklearn.cluster import KMeans

In [84]:

# set number of clusters
k_num_clusters = 5

beijing_grouped_clustering = beijing_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(beijing_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

In [85]:
kmeans.labels_[0:100]

array([4, 4, 3, 4, 1, 0, 4, 4, 0, 1, 0, 4, 4, 1, 4, 0, 0, 4, 4, 4, 1, 3,
       0, 1, 4, 1, 4, 4, 0, 4, 4, 1, 4, 4, 4, 1, 4, 3, 4, 4, 1, 4, 1, 0,
       4, 4, 0, 1, 4, 4, 2, 4, 1, 4, 4, 4, 1, 4, 1, 0, 4, 4, 1, 1, 4, 0,
       1, 4, 4, 4, 0, 1, 0, 1, 1, 1, 4, 4, 0, 4, 4, 4, 4, 1, 3, 1, 4, 0,
       2, 4, 1, 0, 4, 4, 0, 0, 0, 0, 4, 4], dtype=int32)

In [86]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [89]:
beijing_merged = subway_lon_lat.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

beijing_merged.head()

,name,lon,lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,稻田站,116.222049,39.781599,4.0,Botanical Garden,Zoo Exhibit,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
1,东夏园站,116.478299,39.922750,4.0,Hotpot Restaurant,Peking Duck Restaurant,Seafood Restaurant,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Asian Restaurant,Japanese Restaurant,Snack Place
2,蒲黄榆站,116.423826,39.865875,4.0,Fast Food Restaurant,Hotpot Restaurant,Park,Supermarket,Noodle House,Chinese Restaurant,Peking Duck Restaurant,Food & Drink Shop,Food,Flower Shop
3,五道口站,116.337582,39.992683,4.0,Korean Restaurant,Café,Pizza Place,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Bakery,Coffee Shop,Indian Restaurant,Malay Restaurant
4,经海一路站（京东方）,116.546640,39.784002,4.0,Beijing Restaurant,Zoo Exhibit,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


In [90]:
beijing_merged_nonan = beijing_merged.dropna(subset=['Cluster Labels'])

In [91]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [93]:
map_clusters = folium.Map(location=[39.9042, 116.4074], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(beijing_merged_nonan['lat'], beijing_merged_nonan['lon'], beijing_merged_nonan['name'], beijing_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

In [98]:
beijing_merged_nonan.loc[beijing_merged_nonan['Cluster Labels'] == 0, beijing_merged_nonan.columns[[0] + [4] + list(range(5, beijing_merged_nonan.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,马家堡站,Hotel,Supermarket,Hotpot Restaurant,Pizza Place,Sandwich Place,Metro Station,Electronics Store,Exhibit,Food & Drink Shop,Food
11,后沙峪站,Hotel,Market,Zoo Exhibit,Escape Room,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
12,泥洼站,Hotel,Szechuan Restaurant,Metro Station,Vegetarian / Vegan Restaurant,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop
16,巴沟站,Hotpot Restaurant,Department Store,Hotel,Fast Food Restaurant,Zoo Exhibit,Escape Room,Food Court,Food & Drink Shop,Food,Flower Shop
21,四惠东站,Gym,Hotel,Szechuan Restaurant,Metro Station,Zoo Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
28,大葆台站,Department Store,Hotel,Plaza,Fast Food Restaurant,Park,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food
29,大郊亭站,Hotel,Metro Station,Auto Workshop,BBQ Joint,Zoo Exhibit,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food
48,次渠南站,Hotel,Bus Station,Pizza Place,Shopping Mall,Clothing Store,Zoo Exhibit,Exhibit,Food & Drink Shop,Food,Flower Shop
53,南楼梓庄站,Hotpot Restaurant,Convenience Store,Hotel,Auto Workshop,Metro Station,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food
58,大望路站,Chinese Restaurant,Xinjiang Restaurant,Hotel,Japanese Restaurant,Train Station,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market


In [99]:
beijing_merged_nonan.loc[beijing_merged_nonan['Cluster Labels'] == 1, beijing_merged_nonan.columns[[0] + [4] + list(range(5, beijing_merged_nonan.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,牡丹园站,Chinese Restaurant,Café,Pizza Place,Fast Food Restaurant,Seafood Restaurant,BBQ Joint,Asian Restaurant,Hotpot Restaurant,Restaurant,Dongbei Restaurant
15,丰台站,Chinese Restaurant,Shanxi Restaurant,Zoo Exhibit,Escape Room,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
25,国家图书馆站,Chinese Restaurant,African Restaurant,Hotel Pool,Badminton Court,Mongolian Restaurant,Rock Climbing Spot,Gym,Film Studio,Fabric Shop,Farmers Market
33,马连洼站,Chinese Restaurant,Brewery,Asian Restaurant,Park,Fabric Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
36,生命科学园站,Chinese Restaurant,Gym,Italian Restaurant,Coffee Shop,Taiwanese Restaurant,Bakery,Zoo Exhibit,Food Court,Food & Drink Shop,Food
...,...,...,...,...,...,...,...,...,...,...,...
338,陶然亭站,Bakery,Metro Station,Chinese Restaurant,Breakfast Spot,Zoo Exhibit,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
343,车道沟站,Chinese Restaurant,Asian Restaurant,Zoo Exhibit,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
348,褡裢坡站,Metro Station,Chinese Restaurant,Fast Food Restaurant,Zoo Exhibit,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
351,关庄站,Metro Station,Golf Course,Chinese Restaurant,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


In [100]:
beijing_merged_nonan.loc[beijing_merged_nonan['Cluster Labels'] == 2, beijing_merged_nonan.columns[[0] + [4] + list(range(5, beijing_merged_nonan.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,土桥站,Art Gallery,Zoo Exhibit,Exhibit,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
152,崔各庄站,Art Gallery,Zoo Exhibit,Exhibit,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
223,化工站,Street Art,Art Gallery,Zoo Exhibit,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


In [101]:
beijing_merged_nonan.loc[beijing_merged_nonan['Cluster Labels'] == 3, beijing_merged_nonan.columns[[0] + [4] + list(range(5, beijing_merged_nonan.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,肖村站,Furniture / Home Store,Metro Station,Fast Food Restaurant,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
27,石榴庄站,Fast Food Restaurant,Zoo Exhibit,Frozen Yogurt Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
30,草房站,Mobile Phone Shop,Metro Station,Fast Food Restaurant,Café,Zoo Exhibit,Frozen Yogurt Shop,Food Truck,Food Court,Food & Drink Shop,Food
31,梨园站,Metro Station,Department Store,Hotel,Mobile Phone Shop,Fast Food Restaurant,Zoo Exhibit,Fabric Shop,Food Truck,Food Court,Food & Drink Shop
40,八里桥站,Metro Station,Business Service,Train Station,Zoo Exhibit,Fabric Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
66,栗园庄站,Afghan Restaurant,Metro Station,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
104,成寿寺站,Metro Station,Hotel,Chinese Restaurant,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
122,七里庄站,Metro Station,Shopping Mall,Hotpot Restaurant,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
148,丰台南路站,Metro Station,Zoo Exhibit,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
151,清河站,Metro Station,Zoo Exhibit,Exhibit,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio


In [102]:
beijing_merged_nonan.loc[beijing_merged_nonan['Cluster Labels'] == 4, beijing_merged_nonan.columns[[0] + [4] + list(range(5, beijing_merged_nonan.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,稻田站,Botanical Garden,Zoo Exhibit,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
1,东夏园站,Hotpot Restaurant,Peking Duck Restaurant,Seafood Restaurant,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Asian Restaurant,Japanese Restaurant,Snack Place
2,蒲黄榆站,Fast Food Restaurant,Hotpot Restaurant,Park,Supermarket,Noodle House,Chinese Restaurant,Peking Duck Restaurant,Food & Drink Shop,Food,Flower Shop
3,五道口站,Korean Restaurant,Café,Pizza Place,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Bakery,Coffee Shop,Indian Restaurant,Malay Restaurant
4,经海一路站（京东方）,Beijing Restaurant,Zoo Exhibit,Fabric Shop,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
...,...,...,...,...,...,...,...,...,...,...,...
346,永泰庄站,New American Restaurant,Diner,Zoo Exhibit,Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Film Studio
349,车公庄站,Bus Station,Bookstore,Hotel,Coffee Shop,Metro Station,Fast Food Restaurant,Theater,Zoo Exhibit,Fabric Shop,Food Court
350,北土城站,New American Restaurant,Supermarket,Metro Station,Coffee Shop,Shopping Mall,BBQ Joint,Movie Theater,Zoo Exhibit,Fabric Shop,Food & Drink Shop
352,三元桥站,Chinese Restaurant,Clothing Store,Hotel,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,BBQ Joint,Gym / Fitness Center,Sandwich Place,Taiwanese Restaurant
